# Testes de Desempenho

## K-Means

Vamos agora realizar testes de ganho de velocidade de execução, comparando o desempenho do K-Means rodando na CPU com o do K-means rodando na GPU.

Desta vez, iremos utilizar datasets bem maiores e, portanto, nada triviais — como era o caso do [*Iris* Data Set](https://archive.ics.uci.edu/ml/datasets/Iris) que foi usado anteriormente apenas como uma prova de conceito e teste de corretude.

A ideia é testar se os ganhos de desempenho ao utilizarmos uma versão paralelizada em GPU diminuem, estagnam ou aumentam junto com o aumento de instâncias ou dimensionalidade do dataset.



### Dataset 1 (Muito Mais Dimensões)

[TBA]

#### Código

In [2]:
# TBA

### Dataset 2 (Muito Mais Instâncias)

[TBA]

#### Código

In [3]:
# TBA

### Dataset 3 (Muito Mais Insâncias e Dimensões)

Foi utilizado aqui um sub-conjunto dos dados do Dataset **[WESAD (Wearable Stress and Affect Detection)](https://archive.ics.uci.edu/dataset/465/wesad+wearable+stress+and+affect+detection)**, que reúne dados, fisiológicos e de movimento, de diversos sensores presentes em aparelhos *wearables* usados por 15 pacientes diferentes em testes laboratoriais. Um aparelho foi usado no peitoral e outro no pulso dos pacientes.

Esse dataset também contém informações de classe, definindo momentos dos testes como pertencendo à três classificações de emoção do paciente: **referência**, **estresse** ou **diversão**. Portanto, haverão **3 grupos de dados (K = 3)**.

O sub-conjunto de dados utilizado foi: dados obtidos apenas através do **aparelho usado no peito** do paciente, e apenas do **paciente #4**. Utilizando este sub-conjunto, temos **8 variáveis (D = 8)** e **4.588.552 instâncias**, cada uma sendo uma leitura ao longo do tempo do teste laboratorial (leituras realizadas na frequência de 700hz).

Esse sub-conjunto de dados está presente no arquivo `S4/S4_respiban.txt` dentro do arquivo `WESAD.zip` do dataset (também disponível em download direto [neste link](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download)).

#### Código

In [4]:
import kMeans as km
import pandas as pd

import time

import importlib
importlib.reload(km)

# Testing imports
# print(km.kMeansCPU)
# print(km.kMeansGPU)

# Novas variáveis globais
K = 3
MAX_ITERATIONS = 500
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './WESAD/S4/S4_respiban.txt'
columnNames = ['index', 'DI', 'ECG', 'EDA', 'EMG', 'TEMP', 'spatialX', 'spatialY', 'spatialZ', 'RESPIRATION', '_ignore_']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep='\t', index_col=0, skip_blank_lines=True, comment='#')

dataset = dataset.drop(columns=['DI', '_ignore_'])

print(dataset)

           ECG   EDA    EMG   TEMP  spatialX  spatialY  spatialZ  RESPIRATION
index                                                                        
0        34487  2844  32819  27563     37495     32437     31921        33292
1        34274  2869  32481  27560     37485     32433     31935        33295
2        33960  2774  32431  27557     37471     32445     31927        33293
3        33737  2767  32561  27555     37485     32433     31925        33308
4        33602  2768  32696  27562     37487     32429     31909        33300
...        ...   ...    ...    ...       ...       ...       ...          ...
4588548  33272  6470  32721  26727     37539     32597     32256        31863
4588549  33389  6467  32360  26726     37543     32583     32253        31865
4588550  33497  6456  32357  26719     37530     32598     32243        31857
4588551  33499  6450  32175  26733     37539     32585     32263        31855
4588552  33425  6445  32340  26753     37525     32595     32237

In [5]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
              ECG       EDA       EMG      TEMP  spatialX  spatialY  spatialZ   
index                                                                           
0        5.735295  1.508909  4.175522  8.925593  5.800000  5.203079  4.600799  \
1        5.706038  1.527215  3.927640  8.903516  5.789437  5.196481  4.607789   
2        5.662907  1.457652  3.890971  8.881439  5.774648  5.216276  4.603795   
3        5.632276  1.452526  3.986310  8.866721  5.789437  5.196481  4.602796   
4        5.613733  1.453258  4.085316  8.918234  5.791549  5.189883  4.594808   
...           ...       ...       ...       ...       ...       ...       ...   
4588548  5.568405  4.164022  4.103651  2.773508  5.846479  5.467009  4.768057   
4588549  5.584475  4.161826  3.838902  2.766149  5.850704  5.443915  4.766559   
4588550  5.599310  4.153771  3.836701  2.714636  5.836972  5.468658  4.761567   
4588551  5.599585  4.149378  3.703227  2.81766

In [6]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 50

totalExecTime = 0

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    totalExecTime += elapsedTime
    print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f'Final average execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')


Execution time for K-Means CPU run #1: 60.64298701286316
Average execution time for K-Means CPU until now: 60.64298701286316
Execution time for K-Means CPU run #2: 174.71309423446655
Average execution time for K-Means CPU until now: 117.67804062366486
Execution time for K-Means CPU run #3: 152.30988717079163
Average execution time for K-Means CPU until now: 129.22198947270712
Execution time for K-Means CPU run #4: 128.72165322303772
Average execution time for K-Means CPU until now: 129.09690541028976
Execution time for K-Means CPU run #5: 97.57193493843079
Average execution time for K-Means CPU until now: 122.79191131591797
Execution time for K-Means CPU run #6: 96.92578220367432
Average execution time for K-Means CPU until now: 118.4808897972107
Execution time for K-Means CPU run #7: 165.90416526794434
Average execution time for K-Means CPU until now: 125.25564343588692
Execution time for K-Means CPU run #8: 95.03612494468689
Average execution time for K-Means CPU until now: 121.47820

In [7]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 50

totalExecTime = 0

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    totalExecTime += elapsedTime
    print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f'Final average execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')

Execution time for K-Means GPU run #1: 21.983130931854248
Average execution time for K-Means GPU until now: 21.983130931854248
Execution time for K-Means GPU run #2: 15.82835078239441
Average execution time for K-Means GPU until now: 18.90574085712433
Execution time for K-Means GPU run #3: 23.399996995925903
Average execution time for K-Means GPU until now: 20.403826236724854
Execution time for K-Means GPU run #4: 21.378535747528076
Average execution time for K-Means GPU until now: 20.64750361442566
Execution time for K-Means GPU run #5: 16.07504630088806
Average execution time for K-Means GPU until now: 19.73301215171814
Execution time for K-Means GPU run #6: 21.545527696609497
Average execution time for K-Means GPU until now: 20.0350980758667
Execution time for K-Means GPU run #7: 18.654335737228394
Average execution time for K-Means GPU until now: 19.837846313204086
Execution time for K-Means GPU run #8: 46.80391454696655
Average execution time for K-Means GPU until now: 23.20860484

```
Execution time for K-Means GPU run #1: 21.983130931854248
Average execution time for K-Means GPU until now: 21.983130931854248
Execution time for K-Means GPU run #2: 15.82835078239441
Average execution time for K-Means GPU until now: 18.90574085712433
Execution time for K-Means GPU run #3: 23.399996995925903
Average execution time for K-Means GPU until now: 20.403826236724854
Execution time for K-Means GPU run #4: 21.378535747528076
Average execution time for K-Means GPU until now: 20.64750361442566
Execution time for K-Means GPU run #5: 16.07504630088806
Average execution time for K-Means GPU until now: 19.73301215171814
Execution time for K-Means GPU run #6: 21.545527696609497
Average execution time for K-Means GPU until now: 20.0350980758667
Execution time for K-Means GPU run #7: 18.654335737228394
Average execution time for K-Means GPU until now: 19.837846313204086
Execution time for K-Means GPU run #8: 46.80391454696655
Average execution time for K-Means GPU until now: 23.208604842424393
Execution time for K-Means GPU run #9: 33.059858322143555
Average execution time for K-Means GPU until now: 24.30318856239319
Execution time for K-Means GPU run #10: 22.39773440361023
Average execution time for K-Means GPU until now: 24.11264314651489
Execution time for K-Means GPU run #11: 33.869596004486084
Average execution time for K-Means GPU until now: 24.99963886087591
Execution time for K-Means GPU run #12: 27.646646976470947
Average execution time for K-Means GPU until now: 25.22022287050883
Execution time for K-Means GPU run #13: 19.602741241455078
Average execution time for K-Means GPU until now: 24.788108899043156
Execution time for K-Means GPU run #14: 48.29709529876709
Average execution time for K-Means GPU until now: 26.467322213309153
Execution time for K-Means GPU run #15: 28.444780588150024
Average execution time for K-Means GPU until now: 26.599152771631875
Execution time for K-Means GPU run #16: 24.33661937713623
Average execution time for K-Means GPU until now: 26.4577444344759
Execution time for K-Means GPU run #17: 28.94859743118286
Average execution time for K-Means GPU until now: 26.604265198988074
Execution time for K-Means GPU run #18: 25.92827534675598
Average execution time for K-Means GPU until now: 26.566710207197403
Execution time for K-Means GPU run #19: 37.60021781921387
Average execution time for K-Means GPU until now: 27.147421134145635
Execution time for K-Means GPU run #20: 40.94804358482361
Average execution time for K-Means GPU until now: 27.837452256679533
Execution time for K-Means GPU run #21: 17.230555772781372
Average execution time for K-Means GPU until now: 27.33236194792248
Execution time for K-Means GPU run #22: 39.47022771835327
Average execution time for K-Means GPU until now: 27.884083119305696
Execution time for K-Means GPU run #23: 27.823561668395996
Average execution time for K-Means GPU until now: 27.88145175187484
Execution time for K-Means GPU run #24: 36.39567589759827
Average execution time for K-Means GPU until now: 28.236211091279984
Execution time for K-Means GPU run #25: 29.844715118408203
Average execution time for K-Means GPU until now: 28.30055125236511
Execution time for K-Means GPU run #26: 28.696744680404663
Average execution time for K-Means GPU until now: 28.315789461135864
Execution time for K-Means GPU run #27: 20.35076904296875
Average execution time for K-Means GPU until now: 28.020788704907453
Execution time for K-Means GPU run #28: 31.203782558441162
Average execution time for K-Means GPU until now: 28.13446705681937
Execution time for K-Means GPU run #29: 33.52261209487915
Average execution time for K-Means GPU until now: 28.320265161580053
Execution time for K-Means GPU run #30: 34.30063533782959
Average execution time for K-Means GPU until now: 28.519610834121703
Execution time for K-Means GPU run #31: 34.12868905067444
Average execution time for K-Means GPU until now: 28.700548841107278
Execution time for K-Means GPU run #32: 18.91683602333069
Average execution time for K-Means GPU until now: 28.394807815551758
Execution time for K-Means GPU run #33: 38.46372866630554
Average execution time for K-Means GPU until now: 28.699926629210964
Execution time for K-Means GPU run #34: 27.95108151435852
Average execution time for K-Means GPU until now: 28.677901772891772
Execution time for K-Means GPU run #35: 19.63021755218506
Average execution time for K-Means GPU until now: 28.41939650944301
Execution time for K-Means GPU run #36: 20.158329963684082
Average execution time for K-Means GPU until now: 28.189922438727486
Execution time for K-Means GPU run #37: 36.597525119781494
Average execution time for K-Means GPU until now: 28.417154943620837
Execution time for K-Means GPU run #38: 46.41892433166504
Average execution time for K-Means GPU until now: 28.89088571699042
Execution time for K-Means GPU run #39: 19.30056858062744
Average execution time for K-Means GPU until now: 28.64498014939137
Execution time for K-Means GPU run #40: 27.677794218063354
Average execution time for K-Means GPU until now: 28.62080050110817
Execution time for K-Means GPU run #41: 24.5502188205719
Average execution time for K-Means GPU until now: 28.52151802109509
Execution time for K-Means GPU run #42: 26.84169363975525
Average execution time for K-Means GPU until now: 28.48152220249176
Execution time for K-Means GPU run #43: 24.924884796142578
Average execution time for K-Means GPU until now: 28.398809704669688
Execution time for K-Means GPU run #44: 28.81364941596985
Average execution time for K-Means GPU until now: 28.408237879926507
Execution time for K-Means GPU run #45: 21.728301286697388
Average execution time for K-Means GPU until now: 28.259794844521416
Execution time for K-Means GPU run #46: 24.60149121284485
Average execution time for K-Means GPU until now: 28.18026650470236
Execution time for K-Means GPU run #47: 31.33322787284851
Average execution time for K-Means GPU until now: 28.247350789131
Execution time for K-Means GPU run #48: 21.39427423477173
Average execution time for K-Means GPU until now: 28.104578360915184
Execution time for K-Means GPU run #49: 17.096540689468384
Average execution time for K-Means GPU until now: 27.879924530885656
Execution time for K-Means GPU run #50: 27.32244563102722
Average execution time for K-Means GPU until now: 27.86877495288849
Final average execution time for K-Means GPU: 27.86877495288849









Execution time for K-Means CPU run #1: 60.64298701286316
Average execution time for K-Means CPU until now: 60.64298701286316
Execution time for K-Means CPU run #2: 174.71309423446655
Average execution time for K-Means CPU until now: 117.67804062366486
Execution time for K-Means CPU run #3: 152.30988717079163
Average execution time for K-Means CPU until now: 129.22198947270712
Execution time for K-Means CPU run #4: 128.72165322303772
Average execution time for K-Means CPU until now: 129.09690541028976
Execution time for K-Means CPU run #5: 97.57193493843079
Average execution time for K-Means CPU until now: 122.79191131591797
Execution time for K-Means CPU run #6: 96.92578220367432
Average execution time for K-Means CPU until now: 118.4808897972107
Execution time for K-Means CPU run #7: 165.90416526794434
Average execution time for K-Means CPU until now: 125.25564343588692
Execution time for K-Means CPU run #8: 95.03612494468689
Average execution time for K-Means CPU until now: 121.47820362448692
Execution time for K-Means CPU run #9: 141.25038146972656
Average execution time for K-Means CPU until now: 123.67511227395799
Execution time for K-Means CPU run #10: 255.72492051124573
Average execution time for K-Means CPU until now: 136.88009309768677
Execution time for K-Means CPU run #11: 80.07964730262756
Average execution time for K-Means CPU until now: 131.71641620722684
Execution time for K-Means CPU run #12: 80.22452235221863
Average execution time for K-Means CPU until now: 127.42542505264282
Execution time for K-Means CPU run #13: 102.91958141326904
Average execution time for K-Means CPU until now: 125.54036015730638
Execution time for K-Means CPU run #14: 110.97777938842773
Average execution time for K-Means CPU until now: 124.50017581667218
Execution time for K-Means CPU run #15: 86.98605608940125
Average execution time for K-Means CPU until now: 121.99923450152079
Execution time for K-Means CPU run #16: 231.65449333190918
Average execution time for K-Means CPU until now: 128.85268817842007
Execution time for K-Means CPU run #17: 227.33941388130188
Average execution time for K-Means CPU until now: 134.64602498447195
Execution time for K-Means CPU run #18: 119.26533079147339
Average execution time for K-Means CPU until now: 133.7915419737498
Execution time for K-Means CPU run #19: 42.05076456069946
Average execution time for K-Means CPU until now: 128.96308000464188
Execution time for K-Means CPU run #20: 240.94360399246216
Average execution time for K-Means CPU until now: 134.5621062040329
Execution time for K-Means CPU run #21: 127.34345960617065
Average execution time for K-Means CPU until now: 134.2183611279442
Execution time for K-Means CPU run #22: 134.39825654029846
Average execution time for K-Means CPU until now: 134.22653819214213
Execution time for K-Means CPU run #23: 250.14492321014404
Average execution time for K-Means CPU until now: 139.26646797553352
Execution time for K-Means CPU run #24: 80.31269097328186
Average execution time for K-Means CPU until now: 136.8100606004397
Execution time for K-Means CPU run #25: 88.31006932258606
Average execution time for K-Means CPU until now: 134.87006094932556
Execution time for K-Means CPU run #26: 203.58463764190674
Average execution time for K-Means CPU until now: 137.5129292836556
Execution time for K-Means CPU run #27: 264.4283928871155
Average execution time for K-Means CPU until now: 142.21350200970969
Execution time for K-Means CPU run #28: 110.83615708351135
Average execution time for K-Means CPU until now: 141.09288254805975
Execution time for K-Means CPU run #29: 110.77928376197815
Average execution time for K-Means CPU until now: 140.04758603819485
Execution time for K-Means CPU run #30: 141.21603298187256
Average execution time for K-Means CPU until now: 140.08653426965077
Execution time for K-Means CPU run #31: 49.493128299713135
Average execution time for K-Means CPU until now: 137.16416633513666
Execution time for K-Means CPU run #32: 80.10796475410461
Average execution time for K-Means CPU until now: 135.3811600357294
Execution time for K-Means CPU run #33: 179.9872806072235
Average execution time for K-Means CPU until now: 136.732860659108
Execution time for K-Means CPU run #34: 126.99577927589417
Average execution time for K-Means CPU until now: 136.4464759125429
Execution time for K-Means CPU run #35: 118.83537888526917
Average execution time for K-Means CPU until now: 135.94330171176367
Execution time for K-Means CPU run #36: 72.78778958320618
Average execution time for K-Means CPU until now: 134.18898193041483
Execution time for K-Means CPU run #37: 195.48338770866394
Average execution time for K-Means CPU until now: 135.84558749198914
Execution time for K-Means CPU run #38: 141.83007526397705
Average execution time for K-Means CPU until now: 136.0030740123046
Execution time for K-Means CPU run #39: 119.38602900505066
Average execution time for K-Means CPU until now: 135.57699593519553
Execution time for K-Means CPU run #40: 89.28214573860168
Average execution time for K-Means CPU until now: 134.4196246802807
Execution time for K-Means CPU run #41: 95.54292225837708
Average execution time for K-Means CPU until now: 133.4714124260879
Execution time for K-Means CPU run #42: 163.66431283950806
Average execution time for K-Means CPU until now: 134.19029100735983
Execution time for K-Means CPU run #43: 87.57802391052246
Average execution time for K-Means CPU until now: 133.10628479580546
Execution time for K-Means CPU run #44: 95.76155829429626
Average execution time for K-Means CPU until now: 132.25754101168025
Execution time for K-Means CPU run #45: 125.97376489639282
Average execution time for K-Means CPU until now: 132.11790154245165
Execution time for K-Means CPU run #46: 110.78693008422852
Average execution time for K-Means CPU until now: 131.65418477162072
Execution time for K-Means CPU run #47: 80.02452850341797
Average execution time for K-Means CPU until now: 130.55568144676533
Execution time for K-Means CPU run #48: 95.26587152481079
Average execution time for K-Means CPU until now: 129.8204770733913
Execution time for K-Means CPU run #49: 186.57987666130066
Average execution time for K-Means CPU until now: 130.9788321670221
Execution time for K-Means CPU run #50: 72.46529269218445
Average execution time for K-Means CPU until now: 129.80856137752534
Final average execution time for K-Means CPU: 129.80856137752534
           ECG   EDA    EMG   TEMP  spatialX  spatialY  spatialZ  RESPIRATION
index                                                                        
0        34487  2844  32819  27563     37495     32437     31921        33292
1        34274  2869  32481  27560     37485     32433     31935        33295
2        33960  2774  32431  27557     37471     32445     31927        33293
3        33737  2767  32561  27555     37485     32433     31925        33308
4        33602  2768  32696  27562     37487     32429     31909        33300
...        ...   ...    ...    ...       ...       ...       ...          ...
4588548  33272  6470  32721  26727     37539     32597     32256        31863
4588549  33389  6467  32360  26726     37543     32583     32253        31865
4588550  33497  6456  32357  26719     37530     32598     32243        31857
4588551  33499  6450  32175  26733     37539     32585     32263        31855
4588552  33425  6445  32340  26753     37525     32595     32237        31847

[4588553 rows x 8 columns]
##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
              ECG       EDA       EMG      TEMP  spatialX  spatialY  spatialZ   
index                                                                           
0        5.735295  1.508909  4.175522  8.925593  5.800000  5.203079  4.600799  \
1        5.706038  1.527215  3.927640  8.903516  5.789437  5.196481  4.607789   
2        5.662907  1.457652  3.890971  8.881439  5.774648  5.216276  4.603795   
3        5.632276  1.452526  3.986310  8.866721  5.789437  5.196481  4.602796   
4        5.613733  1.453258  4.085316  8.918234  5.791549  5.189883  4.594808   
...           ...       ...       ...       ...       ...       ...       ...   
4588548  5.568405  4.164022  4.103651  2.773508  5.846479  5.467009  4.768057   
4588549  5.584475  4.161826  3.838902  2.766149  5.850704  5.443915  4.766559   
4588550  5.599310  4.153771  3.836701  2.714636  5.836972  5.468658  4.761567   
4588551  5.599585  4.149378  3.703227  2.817661  5.846479  5.447214  4.771552   
4588552  5.589420  4.145716  3.824234  2.964841  5.831690  5.463710  4.758571   

         RESPIRATION  
index                 
0           5.634836  
1           5.635812  
2           5.635161  
3           5.640040  
4           5.637438  
...              ...  
4588548     5.169986  
4588549     5.170636  
4588550     5.168034  
4588551     5.167384  
4588552     5.164781  

[4588553 rows x 8 columns]
```